In [3]:
# Importar Bibliotecas
import os
import pandas as pd
import time
from geopy.geocoders import Nominatim
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring

In [2]:
"""Crea una sesión de Spark con el nombre de la aplicación "Recuperación de Coordenadas de Geolocalización con Nominatim", 
habilitando el soporte para Hive, y la inicializa o reutiliza si ya existe"""
spark = SparkSession \
    .builder \
    .appName("Recuperación de Coordenadas de Geolocalización con Nominatim") \
    .enableHiveSupport() \
    .getOrCreate()

24/10/22 01:27:32 WARN Utils: Your hostname, felipe-Nitro-AN515-44 resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp5s0)
24/10/22 01:27:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/22 01:27:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/22 01:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/22 01:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/22 01:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/10/22 01:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
geo = spark.read.parquet('/home/felipe/despliegue_analytica/files_parquet/geolocation_correios.parquet')

In [ ]:
start_time = time.time()

# Seleccionar ciudades y estados distintos
cidades_ufs = geo.select('cep_prefix', 'city', 'uf').distinct().collect()
qtde_cidades_ufs = geo.select('cep_prefix', 'city', 'uf').distinct().count()

coords = []
counter = 0

# Realiza consultas utilizando Geopy para obtener las coordenadas de cada conjunto de ciudad y estado
for linha in cidades_ufs:
    print(f"{counter}º Consulta de {qtde_cidades_ufs}", end="\r")
    try:
        # Inicializa Nominatim con un tiempo de espera de 30 segundos
        geolocator = Nominatim(user_agent="test_app", timeout=30)
        # Intenta obtener las coordenadas de la ciudad y el estado
        location = geolocator.geocode(f'{linha["city"]}, {linha["uf"]}')
        if location:
            # Si se encuentra la ubicación, agrega la coordenada a la lista
            coords.append([linha['cep_prefix'], location.latitude, location.longitude])
            counter += 1
        else:
            print(f"Ubicación no encontrada para {linha['city']}, {linha['uf']}")
        # Espera 1 segundo entre consultas para no sobrecargar el servicio
        time.sleep(1)
    except Exception as e:
        # Captura y muestra cualquier error ocurrido
        print(f"Error: {e}")
        pass

# Verifica si la lista `coords` contiene datos antes de crear el DataFrame
if len(coords) > 0:
    # Crea un DataFrame con las coordenadas de cada cep
    coords_df = spark.createDataFrame(coords, schema=["cep_prefix", "lat", "lon"])
    
    # Guarda el DataFrame en formato Parquet con el nombre 'geo_coords.parquet'
    coords_df.write.parquet("/home/felipe/despliegue_analytica/files_parquet/geo_coords.parquet")
    print("DataFrame guardado como geo_coords.parquet")
else:
    print("No se obtuvieron coordenadas.")

duration = time.time() - start_time
print(f"Tiempo total: {duration/60} minutos")